<center><h1 style="color:maroon">Unsupervised Learning</h1>
    <img src="figures/07-unsupervised-learning.jpeg" style="width:1300px">
    <h3><span style="color: #045F5F">Data Science & Machine Learning for Planet Earth Lecture Series</span></h3><h6><i> by Cédric M. John <span style="size:6pts">(2022)</span></i></h6></center>

## Plan for today's Lecture 🗓 

* Dimensionality reduction with matrix factorization (PCA)
* Dimensionality reduction with graph algorithms (tSNE, UMAP)
* Clustering algorithms (KMeans, DBSCAN, HDBSCAN)
* Anomaly detection (IFOR, OCSVM)

## Intended learning outcomes 👩‍🎓

* Build a feel for what unsupervised machine learning can do
* Apply dimensionality reduction to data visualization
* Explore dataset by clustering similar observations
* Identify potential outliers

# What is unsupervised learning?


<img src="figures/ml_map.png" style="width:1500px">



<p><strong>Unsupervised Algorithms</strong></p>
<blockquote><p>find patterns in <span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;X&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-9-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-40"><span class="mjx-mrow" id="MJXc-Node-41"><span class="mjx-mi" id="MJXc-Node-42"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.249em; padding-right: 0.024em;">X</span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>X</mi></math></span></span><script id="MathJax-Element-9" type="math/tex">X</script>, <strong>without supervision from a target  <span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-10-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-43"><span class="mjx-mrow" id="MJXc-Node-44"><span class="mjx-mi" id="MJXc-Node-45"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.496em; padding-right: 0.006em;">y</span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>y</mi></math></span></span><script id="MathJax-Element-10" type="math/tex">y</script></strong></p>
</blockquote>


# Dimensionality Reduction


<center><img src="figures/DALLE_eifel.png" style="width:900px;">
 © Cédric John, 2022; Image generated with <a href="https://openai.com/blog/dall-e/">DALL-E</a>
    <br>Prompt: <i>The Eifel Tower contained within a small glass bottle, 3D rendering</i>.</center>

### Dataset

<img src="figures/palmer_logo.png" style="width:500px" align="left"/>

<span style="color:teal">**Today we are using a convenient dataself for visualization:** </span><a href="https://github.com/allisonhorst/palmerpenguins/blob/main/README.md"> the penguins dataset from Palmer Station, Antarctica LTER</a> published in <a href="doi:10.1371/journal.pone.0090081"> Gorman et al, 2014.</a>


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('data/penguins.csv')
data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = data.drop(columns=['species'])
y = data.species
                   
X_train, X_test, y_train_raw, y_test_raw = train_test_split(X, y, train_size=0.8, random_state=12)

label_encoder = LabelEncoder().fit(y_train_raw)

y_train = pd.Series(data=label_encoder.transform(y_train_raw),name='species')
y_test = pd.Series(data=label_encoder.transform(y_test_raw), name='species')

#### Visualizing the relationship between features

In [ ]:
plot_data = X_train.copy(); plot_data['species']=y_train_raw.values;
sns.pairplot(data=plot_data, hue='species');

#### Colinearity

In [ ]:
import matplotlib.pyplot as plt
corr = X_train.corr()
fig, ax = plt.subplots(1,1,figsize=(10, 8))
sns.heatmap(corr, cmap='coolwarm',ax=ax, annot=corr);

# Principal Component Analysis (PCA)
(Dimensionality reduction through matrix factorization)

### Remember Linear Regression Variants<br>
**Polynomial**<br>
$\hat{y} = \beta_0 + \beta_1 X_1 + \beta_2 X_1^2$ <br>
**Log transformation**<br>
$\hat{y} = \beta_0 + \beta_1 \log(X_1)$ <br>


<p>👉 PCA = finding the <strong>best linear combination</strong> of features:</p>


$\color{red}{Z_1} = a_{11} X_1 + a_{12} X_2 + a_{13} X_3$<br>
$\color{red}{Z_2} = a_{21} X_1 + a_{22} X_2 + a_{23} X_3$<br>
$\color{red}{Z_3} = a_{31} X_1 + a_{32} X_2 + a_{33} X_3$ 


<p>👉 <strong>PCA helps to reduce dimensions</strong></p>
<p><img src="figures/lower_dimension_representation.png" width="1200"/></p>
<p>📚 <a href="https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/">Géron, 2017</a></p>


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(random_state=5)
pca.fit(X_train)

# Access our 11 PCs 
W = pca.components_
features = X_train.columns
# Print PCs as COLUMNS
W = pd.DataFrame(W.T,
                 index=features,
                 columns=[f'PC{i}' for i in range(1, 5)])
W


<p> <br/>
☝️ Each PC is a <strong>linear combination of the penguins features</strong></p>


#### Project our original data into the new PCA space

In [ ]:
X_proj_pca = pca.transform(X_train)
X_proj_pca = pd.DataFrame(X_proj_pca, columns=[f'PC{i}' for i in range(1, 5)])
X_proj_pca

### New colinearity:

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 8))
sns.heatmap(X_proj_pca.corr(), cmap='coolwarm',ax=ax, annot=X_proj_pca.corr().astype(int));

### Plotting the data in the PCA space


In [ ]:
import matplotlib.pyplot as plt

# Nice color map:
cm = plt.cm.get_cmap('viridis')

# 2D-slice

plt.figure(figsize=(13,5))
plt.subplot(1,2,1)
plt.title('Before PCA (initial space)'); plt.xlabel('flipper_length_mm'); plt.ylabel('body_mass_g')
plt.scatter(X_train.iloc[:,2], X_train.iloc[:,3],c=y_train, cmap=cm)

plt.subplot(1,2,2)
plt.title('PC1 vs PC2 (new space)'); plt.xlabel('PC 1'); plt.ylabel('PC 2')
plt.scatter(X_proj_pca.iloc[:,0], X_proj_pca.iloc[:,1],c=y_train, cmap=cm);

### PC are ranked by order of importance


In [ ]:
# Let's compute it
X_proj_pca.std()**2 / ((X_proj_pca.std()**2).sum())

In [ ]:
# Sklearn provides it automatically
pca.explained_variance_ratio_


<p>☝️ Automatically ranked by scikit-learn <code>PCA</code></p>



<blockquote><p>68% of the dataset’s variance lies along the first axis</p>
</blockquote>


### PCA for dimensionality reduction

**Choosing 'k'**: <span style = "color:teal">***Trade-off***</span> between compression and performance

**Elbow method**: Look for the <em>inflection point</em> in the explained variance chart. Here, <code>k=2</code> (> 85% of the variance explained) or <code>k=3</code>  (> 90% of the variance explained) are good potential candidates


In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,6))
plt.plot(range(1,5),np.cumsum(pca.explained_variance_ratio_))
plt.ylim(ymin=0.65); plt.title('cumulated share of explained variance', size=16); plt.xlabel('# of principal component used', size=16);


<h4 id="✏️-Test-Model-Performance-(with-k=3-Dimensions)">✏️ Test Model Performance (with <code>k=2</code> Dimensions)</h4>


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# PCA with only 2 components
pca_compressed = PCA(n_components=2,random_state=5)

# Project our data into 2 dimensions
X_train_compressed = pca_compressed.fit_transform(X_train)
X_test_compressed = pca_compressed.transform(X_test)

full_model = KNeighborsClassifier(n_neighbors=8).fit(X_train, y_train)
pca_model = KNeighborsClassifier(n_neighbors=8).fit(X_train_compressed, y_train)

print("Accuracy with 2 PCs")
print(accuracy_score(y_test, pca_model.predict(X_test_compressed)))
print("Accuracy all 4 initial features")
print(accuracy_score(y_test, full_model.predict(X_test)))

### Kernel PCA
Captures non-linear patterns (similar principle to SVM kernels)

In [ ]:
from sklearn.decomposition import KernelPCA

# Fit a PCA with only 2 components
kpca = KernelPCA(kernel='rbf',n_components=2,random_state=5)

X_train_kpca = kpca.fit_transform(X_train)
X_test_kpca = kpca.transform(X_test)

kpca_model = KNeighborsClassifier(n_neighbors=8).fit(X_train_kpca, y_train)

print("Accuracy with 2 PCs")
print(accuracy_score(y_test, kpca_model.predict(X_test_kpca)))

print("\nAccuracy all 4 initial features")
print(accuracy_score(y_test, full_model.predict(X_test)))

## Decompressing PCA
* We can decompressed our compressed data back into 4 dimensions
* Of course, some information will be lost

In [ ]:
# Reconstructed PCA
X_reconstructed = pca_compressed.inverse_transform(X_train_compressed)

# Plotting it
plt.figure(figsize=(15,4)); plt.subplot(1,2,1)
sns.heatmap(X_train.iloc[0:30])
plt.title("original data"); plt.subplot(1,2,2); plt.title("reconstructed data")
sns.heatmap(X_reconstructed[0:30]);

## Limitations of PCA
**PCA cannot separate data on a manifold** (an N-dimensional shape) if they are bent and twisted into a higher dimensional shape: this is a limitation of the projection approach.


<img src="figures/manifold.png" style="width:1500px"/>
<a href="https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/">Hands-On Machine Learning</a>

# Dimensionality reduction and projection using graph algorithms (t-SNE and UMAP)

* Manifold learning based on graph algorithms 
* very complex, general principles exlained only
* More about graph algorithm during the Deep-Learning week

#### In a nutshell:

<img src="figures/umap-only.png" style="width:1000">

## Main Differences between t-SNE and UMAP

#### t-SNE (T-distributed Stochastic Neighbor Embedding)
* Older algorithm
* Relatively simple mathematically: empirical approach
* Based on Gaussian probability and student t-test distribution
* Uses exclusively Euclidian distance between points
* tSNE applies distance normalization

#### UMAP (Uniform Manifold Approximation & Projection)
* More recent algorithm
* Anchored in theoretical mathematical approach
* In order to construct the initial high-dimensional graph, builds a "fuzzy simplicial complex"
* UMAP is often better at preserving global structure in the final projection than T-SNE
* **UMAP is orders of magnitude faster than T-SNE for complex datasets**

**A good first read** if you want to understand the mathematical differences between these two algorithms is this blog by <a href="https://towardsdatascience.com/how-exactly-umap-works-13e3040e1668">Olskolkov, 2019</a>

## t-SNE (T-distributed Stochastic Neighbor Embedding)

A great resource to read about <code>t-SNE</code> is the dedicated blog <a href="https://distill.pub/2016/misread-tsne/">from GoogleBrain</a>. Most of these slides are inspired from it. 

**Key hyperparameters:**
* <code>Perplexity</code> (distance at which points are considered to be linked)

* <code>Perplexity</code> needs to be between 5 and 50, and less than the number of datapoints

<img src="figures/t-sne-hyperparameters.png" style="width:1200">

**Key hyperparameters:**
* <code>Step</code> (number of iterative computation steps)

<img src="figures/t-sne-steps.png" style="width:1200">

* Need sufficient <code>Steps</code> for algorithm to converge!

### What do cluster size and distance 'mean'?

<img src="figures/t-sne-distance.png" style="width:1200">

* Cluster sizes in a t-SNE plot mean nothing (because t-SNE preserves local structure over global structure: large clusters of data tend to be expanded)
* Distances between clusters might not mean anything
* Random noise doesn’t always look random

<img src="figures/t-sne-topology.png" style="width:1200">

* For topology, you may need more than one plot

In [ ]:
from sklearn.manifold import TSNE

TSNE_embedded = TSNE(random_state=11).fit_transform(X_train)
TSNE_embedded.shape


In [ ]:
from utils import compare_embedding
X_proj_PC = X_proj_pca.values
compare_embedding({'Before PCA (initial space)':{'x':X_train.iloc[:,0], 'y':X_train.iloc[:,1],'xlabel':'Culmen Lenght (mm)', 'ylabel':'Culmen Depth (mm'},
 'PCA Projection':{'x':X_proj_PC[:,0], 'y':X_proj_PC[:,1],'xlabel':'PC_1', 'ylabel':'PC_2'},
 'T-SNE Projection':{'x':TSNE_embedded[:,0], 'y':TSNE_embedded[:,1],'xlabel':'TSNE_1', 'ylabel':'TSNE_2'}},labels_data=y_train_raw, marker_size=35)

## UMAP: Uniform Manifold Approximation & Projection

UMAP has some advantages over t-SNE, notably that it preserves the global geometry of the dataset better. However, like t-SNE, hyperparameters selection really matter (and not easy to choose), and the issues are the same as T-SNE (cluster size, distance, and random noise structure might not be meaningful, etc...)


**Key hyperparameters:**
* <code>n_neighbors</code>: The size of local neighborhood (in terms of number of neighboring sample points) used for manifold approximation

* <code>min_distance</code>: The effective minimum distance between embedded points. 

### Example of projection from 3D to 2D

<img src="figures/UMAP-projection.png" style="width:1000">
<a href="https://pair-code.github.io/understanding-umap/">Google Brain</a>

### UMAP preserves global context better than t-SNE 

<img src="figures/t-sne-vs-UMAP2.png" style="width:1000">
<a href="https://pair-code.github.io/understanding-umap/">Google Brain</a>

### On some datasets t-SNE works better than UMAP!

<img src="figures/UMAP-failings.png" style="width:1000">
<a href="https://pair-code.github.io/understanding-umap/">Google Brain</a>

In [ ]:
from umap import UMAP 
umap_trans = UMAP(random_state=11)
X_umap = umap_trans.fit_transform(X_train)

In [ ]:
compare_embedding({'Before PCA (initial space)':{'x':X_train.iloc[:,0], 'y':X_train.iloc[:,1],'xlabel':'Culmen Lenght (mm)', 'ylabel':'Culmen Depth (mm'},
 'PCA Projection':{'x':X_proj_PC[:,0], 'y':X_proj_PC[:,1],'xlabel':'PC_1', 'ylabel':'PC_2'},
 'Unsupervised UMAP Projection':{'x':X_umap[:,0], 'y':X_umap[:,1],'xlabel':'UMAP_1', 'ylabel':'UMAP_2'}
},labels_data=y_train_raw,loc=(-.15,.87),marker_size=35)

# Clustering


<center><img src="figures/DALLE_cows.png" style="width:900px">
 © Cédric John, 2022; Image generated with <a href="https://openai.com/blog/dall-e/">DALL-E</a>
    <br>Prompt: <i>a group of cows all clustered in the middle of a purple prairie, digital art</i>.</center>

### Many clustering algorithms
<a href="https://scikit-learn.org/stable/modules/clustering.html">https://scikit-learn.org/stable/modules/clustering.html</a></p>
<p><img src="figures/sphx_glr_plot_cluster_comparison_001.png" style="width:1200px"/></p>



<p>Find <strong>categories</strong> (classes, segments) of <strong>unlabelled</strong> data rather than just trying to reduce dimensionality</p>
<p>👉 Works better on data that is already clustered, geometrically speaking<br/>
👉 Use PCA for dimensionality reduction beforehand: Euclidean distances work better in lower dimensions)!</p>


## K-Means Explained



<p><img src="figures/K-Means-Clustering-In-Machine-Learning_1.jpg" width="1500px"/></p>
<a href="https://kindsonthegenius.com/blog/what-is-k-means-in-clustering-in-machine-learning/">Kidson, 2017</a>



<p><img src="figures/K-Means-Clustering-In-Machine-Learning_2.jpg" width="1500px"/></p>
<a href="https://kindsonthegenius.com/blog/what-is-k-means-in-clustering-in-machine-learning/">Kidson, 2017</a>



<p><img src="figures/K-Means-Clustering-In-Machine-Learning_3.jpg" width="1500px"/></p>
<a href="https://kindsonthegenius.com/blog/what-is-k-means-in-clustering-in-machine-learning/">Kidson, 2017</a>


### KMeans Loss Function (Inertia)



<p><code>km.fit(X)</code> finds parameters <span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;&amp;#x03B2;&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-22-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-296"><span class="mjx-mrow" id="MJXc-Node-297"><span class="mjx-mi" id="MJXc-Node-298"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.496em; padding-bottom: 0.496em; padding-right: 0.007em;">β</span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi></math></span></span><script id="MathJax-Element-22" type="math/tex">\beta</script> that minimize a loss</p>
<ul>
<li>Each <span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03B2;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-23-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-299"><span class="mjx-mrow" id="MJXc-Node-300"><span class="mjx-msubsup" id="MJXc-Node-301"><span class="mjx-base" style="margin-right: -0.007em;"><span class="mjx-mi" id="MJXc-Node-302"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.496em; padding-bottom: 0.496em; padding-right: 0.007em;">β</span></span></span><span class="mjx-sub" style="font-size: 70.7%; vertical-align: -0.212em; padding-right: 0.071em;"><span class="mjx-mi" id="MJXc-Node-303" style=""><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.496em;">j</span></span></span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>β</mi><mi>j</mi></msub></math></span></span><script id="MathJax-Element-23" type="math/tex">\beta_j</script> parameter is the <strong>centroid</strong> <span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03BC;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-24-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-304"><span class="mjx-mrow" id="MJXc-Node-305"><span class="mjx-msubsup" id="MJXc-Node-306"><span class="mjx-base"><span class="mjx-mi" id="MJXc-Node-307"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.496em;">μ</span></span></span><span class="mjx-sub" style="font-size: 70.7%; vertical-align: -0.212em; padding-right: 0.071em;"><span class="mjx-mi" id="MJXc-Node-308" style=""><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.496em;">j</span></span></span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>μ</mi><mi>j</mi></msub></math></span></span><script id="MathJax-Element-24" type="math/tex">\mu_j</script> of its respective cluster <span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;C&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-25-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-309"><span class="mjx-mrow" id="MJXc-Node-310"><span class="mjx-msubsup" id="MJXc-Node-311"><span class="mjx-base" style="margin-right: -0.045em;"><span class="mjx-mi" id="MJXc-Node-312"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.496em; padding-bottom: 0.311em; padding-right: 0.045em;">C</span></span></span><span class="mjx-sub" style="font-size: 70.7%; vertical-align: -0.212em; padding-right: 0.071em;"><span class="mjx-mi" id="MJXc-Node-313" style=""><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.496em;">j</span></span></span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>C</mi><mi>j</mi></msub></math></span></span><script id="MathJax-Element-25" type="math/tex">C_j</script></li>
</ul>



<ul>
<li>The loss function is called <strong>inertia</strong> <span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;L&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;&amp;#x03BC;&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-26-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-314"><span class="mjx-mrow" id="MJXc-Node-315"><span class="mjx-mi" id="MJXc-Node-316"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.249em;">L</span></span><span class="mjx-mo" id="MJXc-Node-317"><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.434em; padding-bottom: 0.619em;">(</span></span><span class="mjx-mi" id="MJXc-Node-318"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.496em;">μ</span></span><span class="mjx-mo" id="MJXc-Node-319"><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.434em; padding-bottom: 0.619em;">)</span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>L</mi><mo stretchy="false">(</mo><mi>μ</mi><mo stretchy="false">)</mo></math></span></span><script id="MathJax-Element-26" type="math/tex">L(\mu)</script> </li>
</ul>


<ul>
<li>=>  <strong>sum</strong> of <strong>squared distance</strong> between each observation and their <strong>closest centroid:</li> 
<li>$L(\mu) = \sum_{j=1}^{K}\sum_{x_i \in C_j}(||x_i - \mu_j||^2)$</li>
</ul>


## Kmeans in Practice

Algorithm usually run a few times as results depends on initial random seed.

Two <code>sklearn</code> classes:
<ul>
<li><code>scikit.clustering.KMeans</code> </li>
<li><code>scikit.clustering.MiniBatchKMeans</code> — same but uses batch samples instead of the whole dataset, in order to go faster</li>
</ul>


<p>💡Although not mandatory, applying PCA first helps to separate data more easily!</p>


<h3 id="Choosing-Hyperparameter-K">Choosing Hyperparameter K</h3><ul>
<li>Choose <code>K</code> such that the inertia (<code>Kmeans().inertia_</code>) is minimized</li>
<li>We can use the <strong>elbow method</strong> here as well</li>
</ul>


In [ ]:
from sklearn.cluster import KMeans

inertias = []
ks = range(1,10)

for k in ks:
    km_test = KMeans(n_clusters=k).fit(X_proj_pca)
    inertias.append(km_test.inertia_)

plt.plot(ks, inertias)
plt.xlabel('k cluster number');



We can choose either <code>k=2</code> or <code>k=3</code> given the elbow: but <code>k=3</code> seems more reasonable as it gives us a low inertia (plus in our case we do know we have 3 classes so we can cheat!)


## Application of KMeans

In [ ]:

km_pen_pca = KMeans(n_clusters=3, random_state=5)
km_pen_TSNE = KMeans(n_clusters=3, random_state=5)
km_pen_UMAP = KMeans(n_clusters=3, random_state=5)

X_km_pen_pca = km_pen_pca.fit_transform(X_proj_pca)
X_km_pen_TSNE = km_pen_TSNE.fit_transform(TSNE_embedded)
X_km_pen_UMAP = km_pen_UMAP.fit_transform(X_umap)

### KMeans projected in PCA and UMAP Space

In [ ]:
from utils import multi_plot
multi_plot({'K-Means trained on PCA Projection':km_pen_pca.labels_,
             'K-Means trained on UMAP Embedding':km_pen_UMAP.labels_ ,
             'Actual Labels':y_train},encoder=label_encoder, X=pd.DataFrame(X_proj_PC, columns=features), fig_title='PCA Space', marker_size=35);
multi_plot({'K-Means trained on PCA Projection':km_pen_pca.labels_,
             'K-Means trained on UMAP Embedding':km_pen_UMAP.labels_ ,
             'Actual Labels':y_train},encoder=label_encoder, X=pd.DataFrame(X_umap, columns=['UMAP_1','UMAP_2']), fig_title = 'UMAP Space', marker_size=35, X_y_label=('UMAP_1','UMAP_2'));

## DBSCAN (Density-based spatial clustering of applications with noise)

* based on the idea that a cluster in data space is a contiguous region of high point density, separated from other such clusters by contiguous regions of low point density.
* Also uses a distance-based approach to clustering, i.e.  distance to the $k^{th}$ nearest neighbor: <code>k</code> is determined automatically
* find the islands of higher density <code>data</code> amid a sea of sparser <code>noise</code>

<img src="figures/kmeans_vs_dbscan.png" style="width:1000">
<a href="https://github.com/NSHipster/DBSCAN">Matt@GitHub</a>

### Different types of points:
* <code>Core</code> — This is a point that has at least m points within distance n from itself.
* <code>Border</code> — This is a point that has at least one Core point at a distance n.
* <code>Noise</code> — This is a point that is neither a Core nor a Border. And it has less than m points within distance n from itself.

<img src="figures/dbscan_core.png" style="width:800">
<a href="https://www.kdnuggets.com/2020/04/dbscan-clustering-algorithm-machine-learning.html">Chauhan, 2022<a>

### Algorithmic steps for DBSCAN clustering

1. The algorithm proceeds by arbitrarily picking up a point in the dataset (until all points have been visited).
2. If there are at least <code>min_samples</code> points within a radius of ‘ε’ to the point then we consider all these points to be part of the same cluster.
3. The clusters are then expanded by recursively repeating the neighborhood calculation for each neighboring point

<img src="figures/dbscan_in_action.gif" style="width:1000px">
<a href="https://www.kdnuggets.com/2020/04/dbscan-clustering-algorithm-machine-learning.html">Chauhan, 2022<a>

### Key Hyperparameters:

* <code>min_samples</code>: The minimum numper of samples for a cluster to be considered as such. Rule of thumb -> minimum <code>min_samples ≥ D + 1</code>, where <code>D</code> is the dimensions of the dataset. Larger values (<code>min_samples = 2*D)</code> are usually better for data sets with noise and will yield more significant clusters.

* <code>eps</code>: ε (<code>eps</code>) is the maximum distance between two samples for one to be considered as in the neighborhood of the other. In general, small values of ε are preferable, and as a rule of thumb, only a small fraction of points should be within this distance of each other.

* <code>metric</code>: The choice of distance function (<code>metric</code>) is tightly linked to the choice of ε, and has a major impact on the outcomes. There is no estimation for this parameter, but the distance functions need to be chosen appropriately for the data set.

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
# Fit DBSCAN
db_pca = DBSCAN(eps=0.6)
db_pca.fit(X_proj_pca)
db_umap = DBSCAN(eps=0.61)
db_umap.fit(X_umap)

### DBSCAN projected in PCA and UMAP Space

In [ ]:
multi_plot({'DBSCAN trained on PCA Embedding':db_pca.labels_,'DBSCAN trained on Unsupervised UMAP Embedding':db_umap.labels_,'Actual Labels':y_train.values},fig_title='PCA space',encoder = label_encoder, marker_size=35, 
           X=pd.DataFrame(X_proj_PC, columns=features),loc=(.8,-.25));
multi_plot({'DBSCAN trained on PCA Embedding':db_pca.labels_,'DBSCAN trained on Unsupervised UMAP Embedding':db_umap.labels_,'Actual Labels':y_train.values},fig_title = 'UMAP space', encoder = label_encoder, marker_size=35, 
           X=pd.DataFrame(X_umap, columns=['UMAP_1','UMAP_2']), loc=(.7,-.25));

### HDBSCAN (Hierarchical DBSCAN)

* More modern varian of <code>DBSCAN</code> 
* Incorporate some ideas from the algorithm <code>OPTICS</code>
* Uses hierarchical clusters, unlike DBSCAN
* While DBSCAN needs a minimum cluster size and a distance threshold epsilon as user-defined input parameters, HDBSCAN* is basically a DBSCAN implementation for varying epsilon values and therefore only needs the minimum cluster size as single input parameter.

<img src="figures/hdbscan.png" style="width:1000">
Read the paper for more details:<a href="https://link.springer.com/chapter/10.1007/978-3-642-37456-2_14">Campelo et al, 2013</a>

In [ ]:
# Fit HDBSCAN
from hdbscan import HDBSCAN
hdb_pca = HDBSCAN(cluster_selection_method='leaf',cluster_selection_epsilon=0.625)
hdb_pca.fit(X_proj_pca)

### HDBSCAN projected in PCA Space

In [ ]:
multi_plot({'DBSCAN trained on PCA Embedding':db_pca.labels_,'HDBSCAN trained on PCA Embedding':hdb_pca.labels_,'Actual Labels':y_train.values},fig_title = 'PCA space', encoder = label_encoder, marker_size=35, 
           X=pd.DataFrame(X_proj_PC, columns=features),loc=(.8,-.25));

# Outlier (Anomaly) Detection


<center><img src="figures/DALLE_fish.png" style="width:900px;">
 © Cédric John, 2022; Image generated with <a href="https://openai.com/blog/dall-e/">DALL-E</a>
    <br>Prompt: <i>a single red fish isolated from a larger school of fish, digital art</i>.</center>

In [ ]:
X_train_outliers = X_train.copy()

neg_out = X_train_outliers.min()*2
pos_out = X_train_outliers.max()*2

X_train_outliers = X_train_outliers.append([neg_out, pos_out], ignore_index=True)

X_train_outliers

## Once-Class SVM (OCSVM)

One-class SVM is a variation of the SVM that can be used in an unsupervised setting for anomaly detection. There are multiple implementations of this approach (see for instance <a href="https://proceedings.neurips.cc/paper/1999/hash/8725fb777f25776ffa9076e44fcfd776-Abstract.html"> Schölkopt et al, 1999</a>).

<img src="figures/sphx_glr_plot_oneclass_001.png" style="width:1000">
<a href="https://scikit-learn.org/stable/auto_examples/svm/plot_oneclass.html#sphx-glr-auto-examples-svm-plot-oneclass-py">From Scikit-Learn Documentation</a>

* A regular SVM for classification finds a max-margin hyperplane that seperates the positive examples from the negative ones. 

* The one-class SVM simply finds the hyperplane at the center of mass of the dataset: if the data is standardized (zero mean and unit variance) this hyperplane will go through the origin.

* A new hyperparameter, <code>nu</code>, determines the fraction of samples furthest away from the boundary that should be considered as outliers

#### Some of Important Parameters:
* <code>kernel</code>: as with SVM (linear, rbf, etc..)
* <code>nu</code>: Controls the fraction of sample considered as outliers (distance from the margin)

In [ ]:
from sklearn.svm import OneClassSVM

ocsvm = OneClassSVM().fit_predict(X_train_outliers)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12, 8))
ax.scatter(X_train_outliers.culmen_depth_mm, X_train_outliers.flipper_length_mm, c=ocsvm, cmap=cm); 
ax.set_xlabel('culmen_depth_mm', size=16);ax.set_ylabel('flipper_length_mm', size=16);

### Adjusted <code>nu</code> for 5% outliers

In [ ]:
ocsvm = OneClassSVM(nu=0.95 * 0.01 +0.05).fit_predict(X_train_outliers)
fig, ax = plt.subplots(1,1,figsize=(12, 8))
ax.scatter(X_train_outliers.culmen_depth_mm, X_train_outliers.flipper_length_mm, c=ocsvm, cmap=cm); 
ax.set_xlabel('culmen_depth_mm', size=16);ax.set_ylabel('flipper_length_mm', size=16);

## Isolation Forest (IFOR)

Adaptation of the <code>RandomForest</code> classifier.
<img src="figures/ifor1.png" style="width:1200px">
<a href="https://machinelearninggeek.com/outlier-detection-using-isolation-forests/">Pandley, 2020</a>

**Principle:** If data points end up in their own groups closer to the root of the tree, they are more different than the rest of the data: potential outlier.

#### Anomaly Score:
<img src="figures/ifor_score.png">
From <a href="https://engineering.teknasyon.com/what-are-isolation-forests-151d8e98ef5f">Kin, 2022</a>

* h(x): path length of observation x
* c(n): average path length of failed search of binary search tree and number of external nodes 

#### Some of Important Parameters:
* <code>Contamination</code>: It’s the most important parameter of IFOR. The amount of contamination of the data set, i.e. the proportion of outliers in the data set. It defines the threshold on the scores of the samples.
* <code>n_estimators</code>: The number of base estimators in ensemble. Default value is 100.

In [ ]:
from sklearn.ensemble import IsolationForest

ifor = IsolationForest()

In [ ]:
outlier_prediction = ifor.fit_predict(X_train_outliers)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12, 8))
ax.scatter(X_train_outliers.culmen_depth_mm, X_train_outliers.flipper_length_mm, c=outlier_prediction, cmap=cm); 
ax.set_xlabel('culmen_depth_mm', size=16);ax.set_ylabel('flipper_length_mm', size=16);

### Adjusting <code>contamination</code> to reflect 1% of the samples

In [ ]:
ifor_tuned = IsolationForest(random_state=5, contamination=.01)
outlier_prediction2 = ifor_tuned.fit_predict(X_train_outliers)
fig, ax = plt.subplots(1,1,figsize=(12, 8))
ax.scatter(X_train_outliers.culmen_depth_mm, X_train_outliers.flipper_length_mm, c=outlier_prediction2, cmap=cm); 
ax.set_xlabel('culmen_depth_mm', size=16);ax.set_ylabel('flipper_length_mm', size=16);

## Using HDBSCAN <code>outlier_scores_</code>

<code>HDBSCAN</code> is based on the ***density*** of the datapoints in a hypervolume. The distance of each datapoint from a data high-density region can be turned into an <code>outlier score</code>.

In [ ]:
hdbscan_od = HDBSCAN().fit(X_train_outliers)

fig, ax = plt.subplots(1,1,figsize=(12, 8))
cm = plt.cm.get_cmap('RdYlBu')
im = ax.scatter(X_train_outliers.culmen_depth_mm, X_train_outliers.flipper_length_mm,c=hdbscan_od.outlier_scores_, cmap=cm); 
ax.set_xlabel('culmen_depth_mm', size=16);ax.set_ylabel('flipper_length_mm', size=16);
plt.colorbar(im, label='Outlier Score');

## Conclusions on anomaly detection:

*  🧿 Anomaly is a judgement call: an outlier can still be a valid point

*  💣 Whether a point is considered an outlier or not is highly dependent on the model selected and the choice of hyperparameter (**no silver bullet!**)

*  🔮 Always a good idea to combine several methods (**Ensemble**) to determine whether or not the datapoint is truly an outlier.

# Suggested Resources

## 📺 Videos 
* 📼 <a href="https://youtu.be/12Xq9OLdQwQ">Anomaly Detection: Algorithms, Explanations, Applications</a>, long lecture (> 1 hour) by Tom Dietterich, 2018

## 📚 Further Reading 
* 📖 <a href="https://serokell.io/blog/anomaly-detection-in-machine-learning">What Is Anomaly Detection in Machine Learning?</a> by Yulia Gavrilova, 2021
* 📖 <a href="https://pair-code.github.io/understanding-umap/">Understanding UMAP</a> by Andy Coenen and Adam Pearce, Google PAIR
* 📖 <a href="https://towardsdatascience.com/t-sne-clearly-explained-d84c537f53a">t-SNE clearly explained</a> by Kemal Erdem, 2020
